In [99]:
# Stop warnings
# -------------
import warnings
warnings.filterwarnings("ignore")

# General imports
# ---------------
import sys
import multiprocessing
import numpy as np
import scipy.io
import platform
from math import *
import os
import glob
import json
import ipdb
deb = ipdb.set_trace
opj = os.path.join

# MRI analysis imports
# --------------------
import popeye.utilities as utils
from popeye.visual_stimulus import VisualStimulus
import popeye.css_neg as css
import popeye.og_neg as og
import nibabel as nb

# Get inputs
# ----------
cluster_name = 'debug'
fit_model = 'gauss'
subject = 'sub-01'
data_file = '/scratch/mszinte/data/pRFseqTest/pp_data/sub-01/func/sub-01_task-AttendStim_acq-2p5mm_fmriprep_sg_psc_avg.nii.gz'
mask_file = '/scratch/mszinte/data/pRFseqTest/pp_data/sub-01/func/sub-01_task-AttendStim_acq-2p5mm_fmriprep_mask_avg.nii.gz'
slice_nb = 7
opfn = '/scratch/mszinte/data/pRFseqTest/pp_data/sub-01/gauss/fit'

# Define analysis parameters
with open('../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)
    
# Define cluster/server specific parameters
if cluster_name  == 'skylake':
    nb_procs = 32
elif cluster_name  == 'skylake':
    nb_procs = 12
elif cluster_name == 'debug':
    nb_procs = 1
fit_steps = analysis_info["fit_steps"]

In [100]:
# Load data
data_img = nb.load(data_file)
data = data_img.get_fdata()
mask_img = nb.load(mask_file)
mask = mask_img.get_fdata()
slice_mask = mask[:, :, slice_nb].astype(bool)
num_vox = np.sum(slice_mask)
data_slice = data[:,:,slice_nb,:]
data_to_analyse = data_slice[slice_mask]

# determine voxel indices
y, x = np.meshgrid( np.arange(data.shape[1]),np.arange(data.shape[0]))
x_vox,y_vox = x[slice_mask],y[slice_mask]
vox_indices = [(xx,yy,slice_nb) for xx,yy in zip(x_vox,y_vox)]


In [101]:
# Create stimulus design (create in matlab - see others/make_visual_dm.m)
visual_dm_file = scipy.io.loadmat(opj(analysis_info['base_dir'],'pp_data','visual_dm','vis_design.mat'))
visual_dm = visual_dm_file['stim']
stimulus = VisualStimulus(  stim_arr = visual_dm,
                            viewing_distance = analysis_info["screen_distance"],
                            screen_width = analysis_info["screen_width"],
                            scale_factor = 1/10.0,
                            tr_length = analysis_info["TR"],
                            dtype = np.short)

In [102]:
# Initialize model
if fit_model == 'gauss':
    fit_func = og.GaussianFit
    num_est = 6
    model_func = og.GaussianModel(  stimulus = stimulus,
                                    hrf_model = utils.spm_hrf)
elif fit_model == 'css':
    fit_func = css.CompressiveSpatialSummationFit
    num_est = 7
    model_func = css.CompressiveSpatialSummationModel(  stimulus = stimulus,
                                                        hrf_model = utils.spm_hrf)
model_func.hrf_delay = 0
print('models and stimulus loaded')

models and stimulus loaded


In [103]:
# Fit: define search grids (1.5 time stimulus radius)
x_grid = (-15, 15)
y_grid = (-15, 15)
sigma_grid = (0.05, 15)
n_grid =  (0.01, 1.5)

# Fit: define search bounds (3 time stimulus radius)
x_bound = (-30.0, 30.0)
y_bound = (-30.0, 30.0)
sigma_bound = (0.001, 30.0)
n_bound = (0.01, 3)
beta_bound = (-1e3, 1e3)
baseline_bound = (-1e3, 1e3)

if fit_model == 'gauss':
    fit_model_grids =  (x_grid, y_grid, sigma_grid)
    fit_model_bounds = (x_bound, y_bound, sigma_bound, beta_bound, baseline_bound)
elif fit_model == 'css':
    fit_model_grids =  (x_grid, y_grid, sigma_grid, n_grid)
    fit_model_bounds = (x_bound, y_bound, sigma_bound, n_bound, beta_bound, baseline_bound)

In [ ]:
# Fit: main loop
# --------------
print("Slice {slice_nb} containing {num_vox} brain mask voxels".format(slice_nb = slice_nb, num_vox = num_vox))

# Define data
estimates = np.zeros((num_est,data.shape[1]))

# Define multiprocess bundle
bundle = utils.multiprocess_bundle( Fit = fit_func,
                                    model = model_func,
                                    data = data_to_analyse,
                                    grids = fit_model_grids, 
                                    bounds = fit_model_bounds, 
                                    indices = vox_indices, 
                                    auto_fit = True, 
                                    verbose = 1, 
                                    Ns = fit_steps)
# Run fitting
pool = multiprocessing.Pool(processes = nb_procs)
output = pool.map(  func = utils.parallel_fit, 
                    iterable = bundle)

for fit in output:
    estimates[:num_est-1,fit.voxel_index[0]] = fit.estimate
    estimates[num_est-1,fit.voxel_index[0]] = fit.rsquared

Slice 7 containing 30 brain mask voxels
VOXEL=(025,012,007)   TIME=037   RSQ=nan  EST=[  0.7854 278.8973 212.2602      nan      nan]
VOXEL=(022,012,007)   TIME=013   RSQ=0.02  EST=[2.1141 5.8132 0.0313 5.395  0.2647]
VOXEL=(016,016,007)   TIME=037   RSQ=nan  EST=[  0.7854 278.8973 212.2602      nan      nan]
VOXEL=(018,014,007)   TIME=011   RSQ=0.01  EST=[   3.8879   40.8597    7.2801    0.9907 -156.5517]
VOXEL=(023,012,007)   TIME=005   RSQ=0.00  EST=[  0.2308  11.0019   0.4725   1.0101 -38.5134]
VOXEL=(021,012,007)   TIME=009   RSQ=0.02  EST=[1.1642 9.9037 0.0047 1.0635 2.3199]
VOXEL=(019,015,007)   TIME=037   RSQ=nan  EST=[  0.7854 278.8973 212.2602      nan      nan]
VOXEL=(024,013,007)   TIME=037   RSQ=nan  EST=[  0.7854 278.8973 212.2602      nan      nan]
VOXEL=(016,015,007)   TIME=036   RSQ=nan  EST=[  0.7854 278.8973 212.2602      nan      nan]
VOXEL=(017,015,007)   TIME=036   RSQ=nan  EST=[  0.7854 278.8973 212.2602      nan      nan]
VOXEL=(036,010,007)   TIME=037   RSQ=nan 

In [ ]:
# Free up memory
pool.close()
pool.join()

# Save estimates data
new_img = nb.Nifti1Image(dataobj = estimates, affine = data_img.affine, header = data_img.header)
new_img.to_filename(opfn)